# Exam Cheat Detection System Using OpenCV and Deep Learning

## Abstract
Online and computer-based examinations face challenges such as impersonation,
unauthorized collaboration, and use of prohibited materials.  
This project proposes an AI-based exam proctoring system using computer vision
and deep learning to monitor examinee behavior in real time. The system detects
students, analyzes suspicious behaviors over time, and logs cheating evidence
automatically.


In [1]:
import cv2
import time
import csv
import os
from datetime import datetime
from ultralytics import YOLO


In [2]:
# Detection parameters
CONF_THRESHOLD = 0.4
SUSPICIOUS_TIME = 3  # seconds

# Directories
LOG_DIR = "logs"
EVIDENCE_DIR = "evidence"

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(EVIDENCE_DIR, exist_ok=True)

# Color Theme (BGR)
COLOR_NORMAL = (255, 255, 0)        # Cyan
COLOR_LOOKING = (0, 165, 255)       # Orange
COLOR_LEANING = (128, 0, 128)       # Purple
COLOR_CHEATING = (0, 0, 255)        # Red
COLOR_HEADER = (100, 50, 0)         # Dark Blue
COLOR_TEXT = (255, 255, 255)        # White


In [3]:
model = YOLO("yolov8n.pt")


In [4]:
def classify_behavior(x1, y1, x2, y2):
    w = x2 - x1
    h = y2 - y1

    if w > h * 0.9:
        return "Leaning", COLOR_LEANING
    elif h > w * 1.6:
        return "Looking Around", COLOR_LOOKING
    else:
        return "Normal", COLOR_NORMAL


In [5]:
behavior_tracker = {}

def track_behavior(student_id, behavior):
    current_time = time.time()

    if student_id not in behavior_tracker:
        behavior_tracker[student_id] = {
            "behavior": behavior,
            "start": current_time
        }
        return 0

    if behavior_tracker[student_id]["behavior"] == behavior:
        return current_time - behavior_tracker[student_id]["start"]
    else:
        behavior_tracker[student_id] = {
            "behavior": behavior,
            "start": current_time
        }
        return 0


In [6]:
LOG_FILE = os.path.join(LOG_DIR, "cheating_log.csv")

if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "StudentID", "Behavior"])

def log_cheating(frame, student_id, behavior):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    img_path = f"{EVIDENCE_DIR}/{student_id}_{timestamp}.jpg"
    cv2.imwrite(img_path, frame)

    with open(LOG_FILE, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, student_id, behavior])


In [12]:
cap = cv2.VideoCapture(0)
print("Exam Cheat Detection System Using OpenCV and Deep Learning | Press ESC to Exit")

student_id = 1  # simplified single-student ID

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Header bar
    cv2.rectangle(frame, (0, 0), (frame.shape[1], 60), COLOR_HEADER, -1)
    cv2.putText(frame, "Exam Cheat Detection System",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                0.9, COLOR_TEXT, 3)

    results = model(frame, conf=CONF_THRESHOLD, verbose=False)

    for r in results:
        for box in r.boxes:
            if int(box.cls[0]) != 0:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            behavior, color = classify_behavior(x1, y1, x2, y2)
            duration = track_behavior(student_id, behavior)

            label = behavior
            box_color = color

            if duration > SUSPICIOUS_TIME and behavior != "Normal":
                label = f"CHEATING: {behavior}"
                box_color = COLOR_CHEATING
                log_cheating(frame, student_id, behavior)

            # Bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 3)

            # Label background
            (tw, th), _ = cv2.getTextSize(label,
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame,
                          (x1, y1 - th - 12),
                          (x1 + tw + 6, y1),
                          box_color, -1)

            # Label text
            cv2.putText(frame, label,
                        (x1 + 3, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        COLOR_TEXT, 2)

    cv2.imshow("Exam Proctoring", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Exam Cheat Detection System Using OpenCV and Deep Learning | Press ESC to Exit


## Results
The system successfully detects examinees and monitors their behavior in real
time. Suspicious actions sustained over a defined duration are flagged as
cheating and logged with timestamped visual evidence.

## Conclusion
This project demonstrates an effective AI-based approach to automated exam
proctoring using computer vision. The modular design allows easy extension to
include face recognition, eye-gaze tracking, and mobile phone detection.
